In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2022-12-02 18:20:50.691614: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# GPU check
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-12-02 18:20:52.400528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 18:20:52.420129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 18:20:52.420233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# Load data
samples = np.load('rank_samples.npy')
labels = np.load('rank_labels.npy')
print(samples.shape)
print(labels.shape)

(41600, 110, 60)
(41600,)


In [4]:
# Visualize a sample
plt.imshow(samples[0], cmap='gray')

In [5]:
# Shuffle data
indices = np.arange(samples.shape[0])
np.random.shuffle(indices)
samples = samples[indices]
labels = labels[indices]

In [6]:
# Split data
ratio = 0.9
split = int(samples.shape[0] * ratio)
x_train = samples[:split]
y_train = labels[:split]
x_test = samples[split:]
y_test = labels[split:]
print(len(x_train), len(x_test))

37440 4160


In [7]:
# Choose layers
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(110, 60, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(13, activation='softmax')
])

2022-12-02 18:20:52.927616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI AVX512_BF16
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 18:20:52.928000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 18:20:52.928103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 18:20:52.928159: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so 

In [8]:
# Choose loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

In [9]:
# Choose optimizer
opt = 'adam'

In [10]:
# Compile model
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [11]:
# Train model
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


2022-12-02 18:20:54.529405: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303


  15/1170 [..............................] - ETA: 4s - loss: 8.5125 - accuracy: 0.1021    

2022-12-02 18:20:55.480400: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1170/1170 [==============================] - 6s 3ms/step - loss: 0.1776 - accuracy: 0.9693
Epoch 2/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0186 - accuracy: 0.9940
Epoch 3/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0142 - accuracy: 0.9960
Epoch 4/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0115 - accuracy: 0.9963
Epoch 5/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0100 - accuracy: 0.9971
Epoch 6/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0054 - accuracy: 0.9981
Epoch 7/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0084 - accuracy: 0.9976
Epoch 8/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0053 - accuracy: 0.9983
Epoch 9/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0059 - accuracy: 0.9984
Epoch 10/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0041 - accurac

In [12]:
# Evaluate model
model.evaluate(x_test, y_test, verbose=2)

130/130 - 0s - loss: 0.0106 - accuracy: 0.9976 - 250ms/epoch - 2ms/step


[0.010645139962434769, 0.9975961446762085]

In [13]:
# Save model
model.save('rank_model')

INFO:tensorflow:Assets written to: rank_model/assets


INFO:tensorflow:Assets written to: rank_model/assets
